## 모델 클래스 상속
`tf.keras.Model`클래스를 상속하고 자신만의 (forward pass)를 정의하여 커스터마이징된 모델을 만들 수 있음

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # 층을 정의합니다.
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # 정방향 패스를 정의합니다.
    # `__init__` 메서드에서 정의한 층을 사용합니다.
    x = self.dense_1(inputs)
    return self.dense_2(x)

In [7]:
model = MyModel(num_classes=10)

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


ValueError: Specified input shape is not one of the valid types. Please specify a batch input shape of type tuple or list of input shapes. User provided input type: <class 'int'>

## custom layer

custom layer을 만들기 위해 `tf.keras.layers.Layer` 클래스를 상속하고 다음 메서드를 구현한다.
* `__init__`: 이 층에서 사용되는 하위 층을 정의
* `build`: `add_weight`메서드를 사용해 가중치 추가
* `call`: 정방향 패스 구현

In [8]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    # 이 층에서 훈련할 가중치 변수를 만듭니다.
    self.kernel = self.add_weight(name='kernel',
                                  shape=(input_shape[1], self.output_dim),
                                  initializer='uniform',
                                  trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [15]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.build(input_shape=[None, 10])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_layer_6 (MyLayer)         multiple                  100       
_________________________________________________________________
activation_6 (Activation)    multiple                  0         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________
